In [1]:
import numpy as np
from music21 import *
import glob
import sys
import math
from tensorflow.keras.models import Sequential, Model, load_model
sys.path.append("../RL modules/")
from env_noOctave import SegmentationEnv

In [3]:
base_lr = 0.001
max_lr = 0.1
step_size = 8
def masterScheduler(epoch):
    def scheduler():
            period = 2 * step_size
            cycle = math.floor(1 + epoch/period)
            x = abs(epoch/step_size - 2*cycle + 1)
            delta = (max_lr - base_lr)*max(0, (1 - x))
            delta /= float(2 ** (cycle - 1))
            return base_lr + delta
    return scheduler

In [4]:
for i in range(20):
    print(masterScheduler(i)())

0.001
0.013375000000000001
0.025750000000000002
0.038125000000000006
0.0505
0.062875
0.07525000000000001
0.08762500000000001
0.1
0.08762500000000001
0.07525000000000001
0.062875
0.0505
0.038125000000000006
0.025750000000000002
0.013375000000000001
0.001
0.0071875
0.013375000000000001
0.019562500000000003


In [17]:
testing_pieces = []
# for piece in glob.glob('./normal/training/*'):
#     testing_pieces.append(piece)

for piece in glob.glob('./C_normalized/testing/*'):
    testing_pieces.append(piece)

In [21]:
testing_pieces 

'./C_normalized/testing\\trans_Étude_in_Gb_Major_Op25.musicxml'

In [6]:
model = load_model("dqn_normalized")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 48)                1248      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 2,474
Trainable params: 2,474
Non-trainable params: 0
_________________________________________________________________


In [23]:
env = SegmentationEnv([testing_pieces])

./C_normalized/testing\trans_Étude_in_Gb_Major_Op25.musicxml
Total number of pieces 1


In [16]:
len(env.notes),len(testing_pieces)

(1, 1)

In [28]:
def Visualization(piece,offset): #offset assume sorted.
    def cal_offset(e):
        if e is None:
            return 0
        return e.offset + cal_offset(e.activeSite)
    c = converter.parse(piece)
    partStream = c.parts.stream()
    post = c.flattenParts().flat
    all_notes = []
    for note in post.notes:
        print(note,note.offset,note.lyric)
    # for el in post.recurse().notes:
    #     if el.lyric is not None:
    #         el.lyric = el.lyric.replace("♭", "b")
    #     all_notes.append([el.lyric, el, cal_offset(el)])
    # b = sorted(all_notes, key=lambda x: x[0] if x[0] is not None else "ZZZ")
    # b = sorted(b, key=lambda x: x[-1])
    # print(b)
    # for e in b:
    #     if len(offset) >0 and float(round(e[2],1)) >= offset[0]:
    #         del offset[0]
    #         e[1].addLyric("CUT")
    GEX = musicxml.m21ToXml.GeneralObjectExporter(c)
    out = GEX.parse()
    piecename = piece.split("\\")[-1]
    piecename = piecename.split(".")[0]
    piecename = "Vis_"+piecename + ".musicxml"
    print(piece,piecename)
    with open(piecename, "wb") as f:
        f.write(out)

In [29]:
for i in range(len(env.notes)):
    Visualization(env.piecelist[i],[])

<music21.note.Note C> -1.5 None
<music21.chord.Chord G3 C4> -1.0 None
<music21.note.Note B> -0.5 None
<music21.note.Note E> 0.0 GbM(I)
<music21.note.Note E> 0.0 None
<music21.chord.Chord D3 G3 D4> 0.0 None
<music21.chord.Chord C5 E5> 0.25 None
<music21.chord.Chord F4 F5> 0.5 None
<music21.note.Note A> 0.5 None
<music21.chord.Chord F#4 F#5> 0.75 None
<music21.note.Note G> 1.0 V
<music21.note.Note G> 1.0 None
<music21.chord.Chord C3 E3 A3> 1.0 None
<music21.chord.Chord D5 G5> 1.25 None
<music21.chord.Chord A4 A5> 1.5 None
<music21.note.Note G> 1.5 None
<music21.chord.Chord G4 G5> 1.75 None
<music21.note.Note C> 2.0 VI
<music21.note.Note C> 2.0 None
<music21.chord.Chord B2 E3 B3> 2.0 None
<music21.chord.Chord A4 C5> 2.25 None
<music21.chord.Chord D4 D5> 2.5 None
<music21.note.Note F> 2.5 None
<music21.chord.Chord E-4 E-5> 2.75 None
<music21.note.Note E> 3.0 III
<music21.note.Note E> 3.0 None
<music21.chord.Chord C3 F3> 3.0 None
<music21.chord.Chord B4 E5> 3.25 None
<music21.chord.Chord F4

In [ ]:
pieces = ["../musicxml(ok)/Étude_in_Gb_Major_Op25.musicxml"]
env2 = 

In [10]:
for i in range(len(env.notes)):
    offsets = []
    obs = env.reset(i)
    total_reward = 0
    max_reward = 0
    num_correct_segment = 0
    while True:
        obs = obs.reshape((1, 12*2+1))
        action = np.argmax(model.predict(obs))
        offsets.append(env.current_noteoffset)
        obs, reward, done, info = env.step(action)
        #env.render()
        total_reward += reward
        max_reward += 1
        if reward ==1 and action == 1:
            num_correct_segment += 1
        if done:
            break
    print(f"Piece {testing_pieces[i]}, total reward = {total_reward}/{max_reward}, correct segment = {num_correct_segment}")
    print(offset)
    offsets = [float(round(offset,1)) for offset in offsets]
    print(offsets)
#     Visualization(env.piecelist[i],offsets)

Piece ./C_normalized/testing\trans_Il_Vecchio_Castello.musicxml, total reward = 61/205, correct segment = 77


NameError: name 'offset' is not defined